### Objective：

Write an inventory application for computer builds. Basically there will be a pool of inventory (for example 5 x AMD Ryzen 2-2700 CPUs) that can be taken for use, to be added by purchasing or reduced by retiring.

The base class is going to be a general `Resource`. This class should provide functionality common to all the actual resources (CPU, GPU, Memory, HDD, SSD) - for this exercise we're only going to implement CPU, HDD and SSD.

---
### Functionalities and characteristics:

It should provide these at a minimum:

- `name`: user-friendly name of resource instance (e.g. `Intel Core i9-9900K`)
- `manufacturer`: resource instance manufacturer (e.g. `Nvidia`)
- `total`: inventory total (how many are in the inventory pool)
- `allocated`: number allocated (how many are already in use)
- a `__str__` representation that just returns the resource name
- a mode detailed `__repr__` implementation
- `claim(n)`: method to take n resources from the pool (as long as inventory is available)
- `freeup(n)`: method to return n resources to the pool (e.g. disassembled some builds)
- `died(n)`: method to return and permanently remove inventory from the pool (e.g. they broke something) - as long as total available allows it
- `purchased(n)`: method to add inventory to the pool (e.g. they purchased a new CPU)
- `category`: computed property that returns a lower case version of the class name

Next we are going to define child classes for each of CPU, HDD and SDD.

For the `CPU` class:
- `cores` (e.g. `8`)
- `socket` (e.g. `AM4`)
- `power_watts` (e.g. `94`)

For the HDD and SDD classes, we should create an intermediate class called `Storage` with these additional properties:
- `capacity_GB` (e.g. `120`)

The `HDD` class extends `Storage` and has these additional properties:
- `size` (e.g. ``2.5"``)
- `rpm` (e.g. `7000`)

The `SSD` class extends `Storage` and has these additional properties:
- `interface` (e.g. `PCIe NVMe 3.0 x4`)

For all your classes, implement a full constructor that can be used to initialize all the properties, some form of validation on numeric types, as well as customized `__repr__` as you see fit.

For the `total` and `allocated` values in the `Resource` init, think of the arguments there as the **current** total and allocated counts. Those `total` and `allocated` attributes should be private **read-only** properties, but they are modifiable through the various methods such as `claim`, `return`, `died` and `purchased`. Other attributes like `name`, `manufacturer_name`, etc. should be read-only.

---
### Implementation: 

In [1]:
import numbers
import pytest

> Validation & Testing

In [2]:
def validate_integer(arg_name, 
                     arg_value, 
                     min_value=None,
                     max_value=None,
                     custom_min_message=None,
                     custom_max_message=None) -> None:
    """
    Validates that `arg_value` is an interger, and optionally falled within specific bounds. 
    A custom override error message can be provided when min/max bounds are exceeded.

    Args:
        arg_name (str): the name of the argument (used in default error messages)
        arg_value (obj): the value being validated
        min_value (int): optional, specifies the minimum value (inclusive)
        max_value (int): optional, specifies the maximum value (inclusive)
        custom_min_message (str): optional, custom message when value is less than minimum
        custom_max_message (str): optional, custom message when value is greater than maximum

    Returns:
        None: no expections raised if validation passes

    Raises:
        TypeError: if `arg_value` is not an integer
        ValueError: if `arg_value` does not satisfy the bounds
    """
    if not isinstance(arg_value, int):
        raise TypeError('{} must be an integer.'.format(arg_name))

    if min_value is not None and arg_value < min_value:
        if custom_min_message is not None:
            raise ValueError(custom_min_message)
        raise ValueError('{} cannot be less than {}.'.format(arg_name, min_value))

    if max_value is not None and arg_value > max_value:
        if custom_max_message is not None:
            raise ValueError(custom_max_message)
        raise ValueError('{} cannot be greater than {}.'.format(arg_name, max_value))

In [3]:
class TestIntegerValidator:
    
    def test_valid(self):
        validate_integer('arg', 10, min_value=0, max_value=20)
        
    def test_type_error(self):
        with pytest.raises(TypeError):
            validate_integer(1.5)
            
    def test_min_std_error_msg(self):
        with pytest.raises(ValueError) as ex:
            validate_integer('arg', 10, 100)
        assert 'arg' in str(ex.value)
        assert '100' in str(ex.value)
    
    def test_min_custom_error_msg(self):
        with pytest.raises(ValueError) as ex:
            validate_integer('arg', 10, 100, custom_min_message='custom_min_message')
        assert str(ex.value) == 'custom_min_message'
        
    def test_max_std_error_msg(self):
        with pytest.raises(ValueError) as ex:
            validate_integer('arg', 10, 0, 5)
        assert 'arg' in str(ex.value)
        assert '5' in str(ex.value)
    
    def test_max_custom_error_msg(self):
        with pytest.raises(ValueError) as ex:
            validate_integer('arg', 10, 0, 5, custom_max_message='custom_max_message')
        assert str(ex.value) == 'custom_max_message'

In [4]:
tester = TestIntegerValidator()
tester.test_valid()
tester.test_type_error()
tester.test_min_std_error_msg()
tester.test_min_custom_error_msg()
tester.test_max_std_error_msg()
tester.test_max_custom_error_msg()

> Inventory classes (refer to the course instruction for testing part)

In [5]:
class Resource:
    """
    Base class for resources.
    """
    def __init__(self, name, manufacturer, total, allocated):
        """
        
        Args:
            name (str): name of the resource
            manufacturer (str): resource manufacturer
            total (int): current total amount of resources
            allocated (int): current amount of in-use resources (cannot exceed `total`)
        """
        self._name = name
        self._manufacturer = manufacturer
        
        validate_integer('total', total, min_value=0)
        self._total = total
        
        validate_integer('allocated', allocated, min_value=0, max_value=self._total, 
                         custom_max_message='Allocated inventory cannot exceed total inventory.')
        self._allocated = allocated
    
    @property # getter
    def name(self):
        return self._name
    
    @property
    def manufacturer(self):
        return self._manufacturer
    
    @property
    def total(self):
        return self._total
    
    @property
    def allocated(self):
        return self._allocated
    
    @property
    def category(self):
        return type(self).__name__.lower()
    
    @property
    def available(self):
        return self._total - self._allocated
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return '(Name: {} (Category: {}, Manufacturer: {}), \n Total Inventory: {}, \n Allocated Inventory: {})'\
                .format(self.name,
                        self.category,
                        self.manufacturer,
                        self.total,
                        self.allocated)
    
    def claim(self, num):
        """
        Claim `num` amount of inventory items to be taken for use (if available).
        
        Args:
            num (int): number of inventory items to claim
        Returns:
            None
        """
        validate_integer('num', num, 1, self.available, custom_max_message='Cannot claim more than available resources.')
        self._allocated += num
                
    
    def freeup(self):
        """
        Free up `num` amount of inventory items to the pool.
        
        Args:
            num (int): number of inventory items to return
        
        Returns:
            None
        """
        validate_integer('num', num, 1, self.allocated, custom_max_message='Cannot freeup more than allocated resources.')
        self._allocated -= num
    
    def died(self):
        """
        Remove `num` amount of died resources from the inventory pool.
        
        Args:
            num (int): number of inventory items that have died
            
        Returns:
            None
        """
        validate_integer('num', num, 1, self.allocated, custom_max_message='Cannot retire more than allocated.')
        self._total -= num
        self._allocated -= num


    def purchased(self, num):
        """
        Add `num` amount of new inventory items.
        
        Args:
            num (int): number of inventory items to add to the pool
            
        Returns:
            None
        """
        validate_integer('num', num, 1)
        self._total += num

In [6]:
class CPU(Resource):
    """
    Resource subclass used to track specific CPU inventory pools.
    """
    
    def __init__(self, name, manufacturer, total, allocated, cores, socket, power_watts):
        super().__init__(name, manufacturer, total, allocated)
        
        validate_integer('cores', cores, 1)
        self._cores = cores
        self._socket = socket
        validate_integer('power_watts', power_watts, 1)
        self._power_watts = power_watts
        
    @property
    def cores(self):
        """
        Number of cores.
        """
        return self._cores
    
    @property
    def socket(self):
        """
        Socket type of this CPU.
        """
        return self._socket
    
    @property
    def power_watts(self):
        """
        Power watts of this CPU
        """
        return self._power_watts
    
    def __repr__(self):
        return '{}: (CPU Cores: {}, CPU Socket: {}, CPU Powerwatts: {})'.format(self.category,
                                                                                self.cores,
                                                                                self.socket,
                                                                                self.power_watts)

In [7]:
class Storage(Resource):
    """
    A base class for storage devices
    """
    
    def __init__(self, name, manufacturer, total, allocated, capacity_gb):
        super().__init__(name, manufacturer, total, allocated)
        
        validate_integer('capacity_gb', capacity_gb, 1)
        self._capacity_gb = capacity_gb

    @property
    def capacity_gb(self):
        """
        Indicates the capacity (in GB) of the storage device
        """
        return self._capacity_gb

    def __repr__(self):
        return '{}: {} GB'.format(self.category, self.capacity_gb)

In [8]:
class HDD(Storage):
    """
    Class used for HDD type resources
    """
    def __init__(self, name, manufacturer, total, allocated, capacity_gb, size, rpm):
        super().__init__(name, manufacturer, total, allocated, capacity_gb)

        allowed_sizes = ['2.5"', '3.5"']
        if size not in allowed_sizes:
            raise ValueError(f'Invalid HDD size. '
                             f'Must be one of {", ".join(allowed_sizes)}')
        validate_integer('rpm', rpm, min_value=1_000, max_value=50_000)
        self._size = size
        self._rpm = rpm

    @property
    def size(self):
        """
        The HDD size (2.5" / 3.5")
        """
        return self._size

    @property
    def rpm(self):
        """
        The HDD spin speed (rpm)
        """
        return self._rpm

    def __repr__(self):
        s = super().__repr__()
        return f'{s} ({self.size}, {self.rpm} rpm)'

In [9]:
class SSD(Storage):
    """
    Class used for SSD type resources
    """
    def __init__(self, name, manufacturer, total, allocated, capacity_gb, interface):
        super().__init__(name, manufacturer, total, allocated, capacity_gb)

        self._interface = interface

    @property
    def interface(self):
        """
        Interface used by SSD (e.g. PCIe NVMe 3.0 x4)
        """
        return self._interface

    def __repr__(self):
        s = super().__repr__()
        return f'{s} ({self.interface})'